https://horejsek.github.io/python-fastjsonschema/

`pip install fastjsonschema`

In [58]:
import fastjsonschema as fjs

In [104]:
def is_valid(schema, data):
    try:
        validate = fjs.compile(schema)
        return validate(data) == data, None
    except fjs.JsonSchemaException as ex:
        return False, ex

#### compile() is faster

In [59]:
%time fjs.validate({'type': 'string'}, 'hello')

CPU times: user 739 µs, sys: 116 µs, total: 855 µs
Wall time: 865 µs


'hello'

In [60]:
%time fjs.compile({'type': 'string'})('hello')

CPU times: user 462 µs, sys: 72 µs, total: 534 µs
Wall time: 542 µs


'hello'

### basic type

In [105]:
schema = {'type': 'string'}
data = 'hello'

In [106]:
is_valid(schema, data)

(True, None)

In [107]:
schema = {
    'type': 'object',
    'properties': {
        'a': {'type': 'number', 'default': 42},
    },
}
data = {'a': 42}

In [108]:
is_valid(schema, data)

(True, None)

In [109]:
schema = {
    'type': 'object',
    'properties': {
        'a': {'type': 'number', 'default': 42},
    },
}
data = {}

In [110]:
is_valid(schema, data)

(True, None)

In [84]:
schema = {
    '$schema': 'http://json-schema.org/draft-04/schema',
    'type': 'number',
}
data = 100

In [85]:
is_valid(schema, data)

(True, None)

In [86]:
schema = {
    '$schema': 'http://json-schema.org/draft-04/schema',
    'type': 'number',
}
data = 'abc'

In [87]:
is_valid(schema, data)

(False, fastjsonschema.exceptions.JsonSchemaException('data must be number'))

### complex schema: `nested`, `$ref`

https://json-schema.org/understanding-json-schema/structuring.html

In [102]:
schema = {
  "$schema": "http://json-schema.org/draft-07/schema#",

  "definitions": {
    "address": {
      "type": "object",
      "properties": {
        "street_address": { "type": "string" },
        "city":           { "type": "string" },
        "state":          { "type": "string" }
      },
      "required": ["street_address", "city", "state"]
    }
  },

  "type": "object",

  "properties": {
    "name": { "type": "string" },
    "billing_address": { "$ref": "#/definitions/address" },
    "shipping_address": { "$ref": "#/definitions/address" }
  },
  "required": ["name", "billing_address"]
}

data = {
  "name": "Jose Goodman",
  "shipping_address": {
    "street_address": "1600 Pennsylvania Avenue NW",
    "city": "Washington",
    "state": "DC"
  },
  "billing_address": {
    "street_address": "1st Street SE",
    "city": "Washington",
    "state": "DC"
  }
}

In [103]:
is_valid(schema, data)

(True, None)